# GPX Reader - GPS Exchange Format

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import os
import sys

import gpxpy
import numpy as np

import unittest

from file_reader import FileReader

## Main Class

In [2]:
class GpxReader(FileReader):
    '''GPX file - GPS Exchange Format'''

    def __init__(self, filename):
        '''Basic init just records the filename'''

        super().__init__(filename)

        self.points = []


    def load(self):
        '''Load file into memory'''

        with open(self.filename, "rb") as f:
            gpx = gpxpy.parse(f)
            
            counts = \
            {
                'latitude': 0,
                'longitude': 0,
                'elevation': 0,
                'time': 0,
                'speed': 0,
                'course': 0,
                'satellites': 0,
                'horizontal_dilution': 0
            }
    
            for track in gpx.tracks:
                for segment in track.segments:
                    for point in segment.points:
                        self.points.append(point)
                        for item in counts:
                            if getattr(point, item) is not None:
                                counts[item] += 1

            self.numRecords = len(self.points)
            if self.numRecords > 0:
                self.consumePoints(counts)


    def consumePoints(self, counts):
        '''Consume points - convert to standard data structure'''

        # Copy latitude
        iterable = (getattr(point, 'latitude') for point in self.points)
        self.data['lat'] = np.fromiter(iterable, np.double)
            
        # Copy longitude
        iterable = (getattr(point, 'longitude') for point in self.points)
        self.data['lon'] = np.fromiter(iterable, np.double)
            
        # Copy elevation, if present
        if counts['elevation'] > 0:
            iterable = (getattr(point, 'elevation') for point in self.points)
            self.data['ele'] = np.fromiter(iterable, np.single)

        # Copy timestamp, if present
        if counts['time'] > 0:
            iterable = (getattr(point, 'time').timestamp() for point in self.points)
            self.data['timestamp'] = np.fromiter(iterable, np.double)
        
        # Copy SOG, if present
        if counts['speed'] > 0:
            iterable = (getattr(point, 'speed') for point in self.points)
            self.data['sog'] = np.fromiter(iterable, np.double)

        # Copy COG, if present - COROS uses <cog>...</cog> which is incorrect and is not supported by gpxpy
        if counts['course'] > 0:
            iterable = (getattr(point, 'course') for point in self.points)
            self.data['cog'] = np.fromiter(iterable, np.double)

        # Copy satellites, if present with logic to handle <sat></sat>
        if counts['satellites'] > 0:
            iterable = (0 if getattr(point, 'satellites') is None else getattr(point, 'satellites') for point in self.points)
            self.data['sat'] = np.fromiter(iterable, np.uint8)

        # Copy HDOP, if present
        if counts['horizontal_dilution'] > 0:
            iterable = (getattr(point, 'horizontal_dilution') for point in self.points)
            self.data['hdop'] = np.fromiter(iterable, np.double)

## Unit Tests

In [3]:
class TestData(unittest.TestCase):
    '''Class to test data was correctly loaded'''

    def testNumRecords(self):
        '''Test the number of records is as expected'''

        self.assertEqual(gpxReader.numRecords, 3809)

        for fieldName in gpxReader.data:
            self.assertEqual(gpxReader.data[fieldName].size, 3809)


    def testHdop(self):
        '''Test the horizontal dilution of precision is as expected'''

        self.assertEqual(gpxReader.data['hdop'].min(), 0.7)
        self.assertEqual(gpxReader.data['hdop'].max(), 1.6)


    def testSat(self):
        '''Test the satellite count is as expected'''

        self.assertEqual(gpxReader.data['sat'].min(), 0)
        self.assertEqual(gpxReader.data['sat'].max(), 14)


    def testTimestamp(self):
        '''Test the timestamp is as expected'''

        self.assertEqual(gpxReader.data['timestamp'].min(), 1649672017.0)
        self.assertEqual(gpxReader.data['timestamp'].max(), 1649677777.0)


    def testLat(self):
        '''Test the latitude is as expected'''

        self.assertEqual(gpxReader.data['lat'].min(), 50.5705683)
        self.assertEqual(gpxReader.data['lat'].max(), 50.5832829)


    def testLon(self):
        '''Test the longitude is as expected'''

        self.assertEqual(gpxReader.data['lon'].min(), -2.4620381)
        self.assertEqual(gpxReader.data['lon'].max(), -2.4559603)


    def testEle(self):
        '''Test the elevation is as expected'''

        if 'ele' in gpxReader.data:
            self.assertEqual(gpxReader.data['ele'].min(), -3.169 )
            self.assertEqual(gpxReader.data['ele'].max(), 13.548)


    def testCog(self):
        '''Test the course over ground is as expected'''

        if 'cog' in gpxReader.data:
            self.assertEqual(gpxReader.data['cog'].min(), 0.00056)
            self.assertEqual(gpxReader.data['cog'].max(), 359.99618)

In [4]:
if __name__ == '__main__':
    # Determine whether session is interactive or batch to facilitate unittest.main(..., exit=testExit)
    import __main__ as main
    testExit = hasattr(main, '__file__')

    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    filename = os.path.join(projdir, 'sessions', '20220411', 'APEX_Pro_Speedsurfing20220411111317.gpx')
    gpxReader = GpxReader(filename)
    gpxReader.load()

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

........
----------------------------------------------------------------------
Ran 8 tests in 0.004s

OK
